<a href="https://colab.research.google.com/github/boothmanrylan/MSSForestDisturbances/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports and installs

In [6]:
# standard python imports
import os

In [7]:
# earth engine api import and authentication
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWg5MGrGK928eym9BQYUJzMa0g6NSmQmO0fYGzcorp2A7bgkkA0KosY



Successfully saved authorization token.


In [8]:
# pip installable packages
try:
    import geemap
except:
    !pip install geemap
    import geemap

In [9]:
# autoreload all python modules before executing code block
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# install and import my packages
this_dir = os.getcwd()
msslib_dir = '~/school/msslib/'
sequencer_dir = '~/school/sequencer/'
mfd_dir = '~/school/MSSForestDisturbances/'

packages = [msslib_dir, sequencer_dir, mfd_dir]

for package in packages:
    %cd {package}
    ! pip install .
    
%cd {this_dir}
    
from msslib import msslib
import sequencer
import mfd

/home/rylan/school/msslib
Processing /home/rylan/school/msslib
  Preparing metadata (setup.py) ... done
  Created wheel for msslib: filename=msslib-0.1.0-py3-none-any.whl size=13506 sha256=bf35b80cc7357ce04e1283e2236164e99112ae9f09716b9c993a82a96bc44e22
  Stored in directory: /tmp/pip-ephem-wheel-cache-1q3_l4qa/wheels/62/0f/de/7375dcf731b582a190c62c301d1bae8fbf24650f27285fa167
Successfully built msslib
  Attempting uninstall: msslib
    Found existing installation: msslib 0.1.0
    Uninstalling msslib-0.1.0:
      Successfully uninstalled msslib-0.1.0
You should consider upgrading via the '/home/rylan/.envs/tf/bin/python -m pip install --upgrade pip' command.
/home/rylan/school/sequencer
Processing /home/rylan/school/sequencer
  Preparing metadata (setup.py) ... done
  Created wheel for sequencer: filename=sequencer-0.0.0-py3-none-any.whl size=4214 sha256=5a48dfc9b11dfc46c2be24567582db9c38c38cb37e3317300e8e6aad4fa39461
  Stored in directory: /tmp/pip-ephem-wheel-cache-ny4jj4bc/wheels/8

# Test

In [11]:
def get_valid_burns(events):
    years = events.distinct("year").aggregate_array("year")

    def _valid(year):
        curr_events = events.filter(ee.Filter.eq("year", year))
        exclusively_burnt = curr_events.reduce(ee.Reducer.allNonZero())
        burnt_often = curr_events.reduce(ee.Reducer.sum()).gt(1)
        return exclusively_burnt.Or(burnt_often).selfMask()

    result = ee.ImageCollection.fromImages(years.map(_valid))
    return result.reduce(ee.Reducer.anyNonZero())

def postprocess(event):
    event = mfd.postprocess.postprocess(event)
    event = mfd.postprocess.squash_extra_classes(event)
    event = mfd.postprocess.set_true_date(event)
    return event

In [12]:
grid_split_qc = ee.FeatureCollection('users/boothmanrylan/gridSplitForestedQC')

grid_cell = grid_split_qc.filter(ee.Filter.eq("id", 48)).first().geometry()


In [13]:
quebec_fires = ee.ImageCollection("users/boothmanrylan/MSSForestDisturbances/test2")

events = mfd.postprocess.filter_and_clip(quebec_fires, grid_cell)

processed_events = events.map(postprocess)

events_carried_forward = mfd.postprocess.carry_observations_forward(processed_events)
burns = sequencer.match_sequence(events_carried_forward, ee.List([1, 1, 1]))

permanent_burns = mfd.postprocess.get_burn_year_spread(processed_events).gt(5)

valid_burns = get_valid_burns(processed_events).And(permanent_burns.Not())
all_burns = processed_events.reduce(ee.Reducer.anyNonZero())

In [14]:
binary_vis = {'min': 0, 'max': 1}

Map = geemap.Map()
Map.addLayer(grid_split_qc, {}, 'Grid Split QC')
Map.addLayer(grid_cell, {}, 'Grid Cell of Interest')
Map.addLayer(valid_burns, binary_vis, 'Valid Burns')
Map.addLayer(all_burns, binary_vis, 'All Burns')
Map.addLayer(burns, binary_vis, "Burns")
Map.centerObject(grid_cell, 10)

Map

Map(center=[51.66505765621107, -78.15572978972988], controls=(WidgetControl(options=['position', 'transparent_…